In [2]:
!pip install music21

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.8/22.8 MB 71.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.4/199.4 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 2.7 MB/s eta 0:00:00


In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/lofi-hip-hop-midi/Piano 2 MIDI.mid
/kaggle/input/lofi-hip-hop-midi/Cymatics - Lofi MIDI 20 - G Min.mid
/kaggle/input/lofi-hip-hop-midi/Lofi Piano MIDI.mid
/kaggle/input/lofi-hip-hop-midi/Cymatics - Eternity MIDI 22 - B Min.mid
/kaggle/input/lofi-hip-hop-midi/Piano MIDI (3).mid
/kaggle/input/lofi-hip-hop-midi/Cymatics - Lofi MIDI 14 - F Min.mid
/kaggle/input/lofi-hip-hop-midi/5.mid
/kaggle/input/lofi-hip-hop-midi/Cymatics - Lofi MIDI 1 - C Maj.mid
/kaggle/input/lofi-hip-hop-midi/3.mid
/kaggle/input/lofi-hip-hop-midi/E-Piano MIDI (2).mid
/kaggle/input/lofi-hip-hop-midi/1.mid
/kaggle/input/lofi-hip-hop-midi/Rhodes MIDI (4).mid
/kaggle/input/lofi-hip-hop-midi/Piano MIDI (4).mid
/kaggle/input/lofi-hip-hop-midi/20.mid
/kaggle/input/lofi-hip-hop-midi/Piano Chords MIDI (3).mid
/kaggle/input/lofi-hip-hop-midi/Cymatics - Lofi MIDI 16 - F Maj.mid
/kaggle/input/lofi-hip-hop-midi/Cymatics - Eternity MIDI 3 - D Maj.mid
/kaggle/input/lofi-hip-hop-midi/Cymatics - Eternity MIDI 14 - A Min

In [4]:
import glob
from music21 import converter, instrument, note, chord

def get_notes_from_midi(midifiles_path):
    notes = []

    for file in glob.glob(midifiles_path + "/*.mid"):
        midi = converter.parse(file)

        print(f"Parsing {file}")

        notes_to_parse = None

        try:  # File has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse()
        except:  # File has notes in flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                # Convert chord into a tuple of note names
                notes.append('.'.join(str(n) for n in element.normalOrder))

    return notes

# Example usage
notes = get_notes_from_midi("/kaggle/input/lofi-hip-hop-midi")
print(f"Collected {len(notes)} notes/chords.")

Parsing /kaggle/input/lofi-hip-hop-midi/Piano 2 MIDI.mid
Parsing /kaggle/input/lofi-hip-hop-midi/Cymatics - Lofi MIDI 20 - G Min.mid
Parsing /kaggle/input/lofi-hip-hop-midi/Lofi Piano MIDI.mid
Parsing /kaggle/input/lofi-hip-hop-midi/Cymatics - Eternity MIDI 22 - B Min.mid
Parsing /kaggle/input/lofi-hip-hop-midi/Piano MIDI (3).mid
Parsing /kaggle/input/lofi-hip-hop-midi/Cymatics - Lofi MIDI 14 - F Min.mid
Parsing /kaggle/input/lofi-hip-hop-midi/5.mid
Parsing /kaggle/input/lofi-hip-hop-midi/Cymatics - Lofi MIDI 1 - C Maj.mid
Parsing /kaggle/input/lofi-hip-hop-midi/3.mid
Parsing /kaggle/input/lofi-hip-hop-midi/E-Piano MIDI (2).mid
Parsing /kaggle/input/lofi-hip-hop-midi/1.mid
Parsing /kaggle/input/lofi-hip-hop-midi/Rhodes MIDI (4).mid
Parsing /kaggle/input/lofi-hip-hop-midi/Piano MIDI (4).mid
Parsing /kaggle/input/lofi-hip-hop-midi/20.mid
Parsing /kaggle/input/lofi-hip-hop-midi/Piano Chords MIDI (3).mid
Parsing /kaggle/input/lofi-hip-hop-midi/Cymatics - Lofi MIDI 16 - F Maj.mid
Parsing /k

In [5]:
from collections import defaultdict
import numpy as np
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

# Encode the notes
encoder = LabelEncoder()
encoded_notes = encoder.fit_transform(notes)
n_vocab = len(set(encoded_notes))

sequence_length = 100  # You can experiment with this

# Create input sequences and the corresponding outputs
network_input = []
network_output = []

for i in range(len(encoded_notes) - sequence_length):
    seq_in = encoded_notes[i:i + sequence_length]
    seq_out = encoded_notes[i + sequence_length]
    network_input.append(seq_in)
    network_output.append(seq_out)

n_patterns = len(network_input)
print(f"Total patterns: {n_patterns}")

# Reshape and normalize input
network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
network_input = network_input / float(n_vocab)

# One-hot encode the output
network_output = to_categorical(network_output, num_classes=n_vocab)

Total patterns: 1800


In [6]:
network_output[1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [7]:
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense, Activation

def create_model(network_input, n_vocab):
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
    model.add(Dropout(0.1))
    model.add(LSTM(512, return_sequences=True))
    model.add(Dropout(0.1))
    model.add(LSTM(512))
    model.add(Dense(256))
    model.add(Dropout(0.1))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    return model

model = create_model(network_input, n_vocab)
model.summary()

/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 100, 512)       │     1,052,672 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 100, 512)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 100, 512)       │     2,099,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 100, 512)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 512)            │     2,099,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 254)            │        65,278 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 254)            │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,447,678 (20.78 MB)

 Trainable params: 5,447,678 (20.78 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

RuntimeError: Physical devices cannot be modified after being initialized

In [12]:
# Train the model
model.fit(network_input, network_output, epochs=200, batch_size=64)

Epoch 1/200
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 77ms/step - loss: 4.4690
Epoch 2/200
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 75ms/step - loss: 4.5986
Epoch 3/200
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 76ms/step - loss: 4.3483
Epoch 4/200
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 76ms/step - loss: 4.2189
Epoch 5/200
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 76ms/step - loss: 4.1095
Epoch 6/200
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 77ms/step - loss: 4.0154
Epoch 7/200
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 76ms/step - loss: 4.0530
Epoch 8/200
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 77ms/step - loss: 3.8042
Epoch 9/200
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 77ms/step - loss: 3.6604
Epoch 10/200
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 77ms/step - loss: 3.6467
Epoch 11/200
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 78ms/step - loss: 3.5463
Epoch 12/200
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 78ms/step - loss: 3.4428
Epoch 13/200
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 78ms/step - loss: 3.1642
Epoch 14/200
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 79ms/step - loss: 3.0433
Epoch 15/200
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 78ms/step - lo

In [14]:
import random
import sys
from music21 import stream, instrument, note, chord

# Create a dictionary to map integers back to notes
int_to_note = dict((number, note) for number, note in enumerate(encoder.classes_))

# Pick a random sequence from the input as a starting point
start = np.random.randint(0, len(network_input)-1)
pattern = network_input[start]
pattern = pattern[:,0]  # Remove the scaling
pattern = [int_to_note[int(value * n_vocab)] for value in pattern]

# Generate notes
generated_notes = []

for i in range(50):  # Number of notes to generate
    # Prepare input
    input_seq = [encoder.transform([n])[0] for n in pattern]
    input_seq = np.reshape(input_seq, (1, len(input_seq), 1))
    input_seq = input_seq / float(n_vocab)

    # Predict
    prediction = model.predict(input_seq, verbose=0)
    index = np.argmax(prediction)
    result = int_to_note[index]
    generated_notes.append(result)

    # Append to the pattern
    pattern.append(result)
    pattern = pattern[1:]

# Convert to MIDI
def create_midi(generated_notes, output_filename="output.mid"):
    offset = 0
    output_notes = []

    for pattern in generated_notes:
        # Chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for n in notes_in_chord:
                new_note = note.Note(int(n))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        else:
            # Single note
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Guitar()
            output_notes.append(new_note)

        offset += 0.5  # Adjust as needed

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp=output_filename)

create_midi(generated_notes, "generated_music.mid")
print("Music generation complete! MIDI file saved as 'generated_music.mid'")

Music generation complete! MIDI file saved as 'generated_music.mid'
